### Declare common params

In [1]:
base_path="../eit_basic_dual_unreduced_experiments/exp_1" #Don't add '/' at start and end
T=200 #Training time period
C=100000 #Capital available
file=1 #Index_file
m=8 #Inital Size of Kernel
lbuck=2 #Size Of Buckets
pho=0.1 #Transaction Cost Proportion
lamda=1/(100*C) # lower_bound for capital invested in jth stock
nuh=0.6  # upper_bound
xii=1.2 # Limit for Tracking Error
k=20 #Cardinality constraint for stocks in portfolio
p=3 #If stock not selected in optimal soln in last p iters then it is to be dropped from kernel
f=12
w_return=100
w_risk=110
w_risk_down=1
#cj,cb are declared in script

In [2]:
import sys
sys.path.insert(0,"..") #Add root folder to sys path

from eit_basic_experiment import TestEitBasic
from eit_dual_exp import TestEitDual

import time
from tqdm.notebook import tqdm
from IPython.utils import io
from joblib import Parallel, delayed
import pandas as pd

### Generate instances of test objects for various combinations of params

In [3]:
!rm -r $base_path

In [4]:
path_pattern=base_path+"/{}/file={},pho={},xii={},k={},m={},lbuck={},p={},nuh={}"

In [5]:
%%time
basic_instances=[]
dual_instances=[]
for file in [1]:
    for pho in [0.05,0.1,0.2]:
        for xii in [1.2,1.3,1.4]:
            for k in [12,16,25]:
                for m in [8,12,16]:
                    for nuh in [0.3,0.45,0.6]:
                        output_basic=path_pattern.format("basic",file,pho,xii,k,m,lbuck,p,nuh)
                        #Create object for EIT-basic
                        test_eit_basic=TestEitBasic(output=output_basic,T=T,C=C,file=file,m=m,lbuck=lbuck,\
                                            pho=pho,lamda=lamda,nuh=nuh,xii=xii,k=k,p=p,f=f)
                        basic_instances.append(test_eit_basic)
                        #Create object for EIT-dual
                        output_dual=path_pattern.format("dual",file,pho,xii,k,m,lbuck,p,nuh)
                        test_eit_dual=TestEitDual(output=output_dual,T=T,C=C,file=file,m=m,lbuck=lbuck,pho=pho,\
                                                  lamda=lamda,nuh=nuh,xii=xii,k=k,p=p,f=f,w_risk=w_risk,\
                                                  w_return=w_return,w_risk_down=w_risk_down)
                        dual_instances.append(test_eit_dual)

CPU times: user 3.73 ms, sys: 658 µs, total: 4.39 ms
Wall time: 4.05 ms


### Run the EIT problems with threading

In [6]:
def parallel_func(eit_object):
    params=eit_object.give_params()
    step_dict={}
    try:
        result_df=eit_object.run_experiment(from_root=False,verbose=False);
        result_df["z_linear"]=eit_object.objective_linear
        #result_dict.update(step_dict)
    except Exception as e:
        print("Error for params",params)
        result_df=None
    step_dict[params]=result_df   
    return (step_dict)

In [7]:
%%time
#test_2={}
basic_outputs=Parallel(verbose = 2,n_jobs  = -1\
            )(delayed( parallel_func )(run) for run in tqdm(basic_instances))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  7.2min



CPU times: user 844 ms, sys: 150 ms, total: 994 ms
Wall time: 11min 41s


[Parallel(n_jobs=-1)]: Done 243 out of 243 | elapsed: 11.7min finished


In [8]:
%%time
#test_2={}
dual_outputs=Parallel(verbose = 2,n_jobs  = -1\
            )(delayed( parallel_func )(run) for run in tqdm(dual_instances))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  5.5min
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed: 26.8min



CPU times: user 922 ms, sys: 107 ms, total: 1.03 s
Wall time: 42min 44s


[Parallel(n_jobs=-1)]: Done 243 out of 243 | elapsed: 42.7min finished


### Aggregate results

In [12]:
def make_result(parallel_output):
    final_result=pd.DataFrame()
    for output_i in parallel_output:
        params,result_df=next(iter(output_i.items()))
        param_part=pd.DataFrame([params])
        if result_df is None:
            res_part=pd.DataFrame()
        else:
            res_part=pd.DataFrame([result_df.loc[result_df.z_value.idxmax()]])
        temp=pd.concat([param_part,res_part.reset_index()],axis=1,ignore_index=False)
        final_result=final_result.append(temp)
    return (final_result)

In [10]:
basic_result=make_result(basic_outputs)
basic_result.to_csv(base_path+"/basic_results_exp_1.csv")

In [11]:
dual_result=make_result(dual_outputs)
dual_result.to_csv(base_path+"/dual_results_exp_1.csv")

### Compare results

In [13]:
basic_result.columns

Index(['output', 'T', 'C', 'file', 'm', 'lbuck', 'pho', 'lamda', 'nuh', 'xii',
       'k', 'p', 'f', 'index', 'bucket', 'kernel_size', 'problem_status',
       'z_value', 'in_excess_return', 'in_tr_err', 'out_excess_return',
       'out_tr_err', 'portfolio_size', 'z_linear'],
      dtype='object')

In [17]:
print("Basic Infeasible for % of param combs={:.2f}",basic_result.z_value.isnull().mean())
print("Dual Infeasible for % of param combs={:.2f}",dual_result.z_value.isnull().mean())

Basic Infeasible for % of param combs={:.2f} 0.3004115226337449
Dual Infeasible for % of param combs={:.2f} 0.04526748971193416


In [20]:
basic_result["return_pu_risk_sample"]=basic_result["in_excess_return"]/basic_result["in_tr_err"].abs()
basic_result["return_pu_risk_oot"]=basic_result["out_excess_return"]/basic_result["out_tr_err"].abs()

dual_result["return_pu_risk_sample"]=dual_result["in_excess_return"]/dual_result["in_tr_err"].abs()
dual_result["return_pu_risk_oot"]=dual_result["out_excess_return"]/dual_result["out_tr_err"].abs()

In [23]:
basic_k_pho_nuh=basic_result.groupby(["k","pho","nuh"])[["return_pu_risk_sample","return_pu_risk_oot"]].mean()
dual_k_pho_nuh=dual_result.groupby(["k","pho","nuh"])[["return_pu_risk_sample","return_pu_risk_oot"]].mean()

In [24]:
from IPython.display import display_html 

In [25]:
df1_styler = basic_k_pho_nuh.style.set_table_attributes("style='display:inline'").set_caption('EIT Basic')
df2_styler = dual_k_pho_nuh.style.set_table_attributes("style='display:inline'").set_caption('EIT Dual')

display_html(df1_styler._repr_html_()+df2_styler._repr_html_(), raw=True)

EIT Basic return_pu_risk_sample return_pu_risk_oot k pho nuh 
 
 12 
 0.05 
 0.3 
 0.260423 
 -0.002815 
 
 
 0.45 
 nan 
 nan 
 
 
 0.6 
 0.256674 
 -0.015257 
 
 
 0.1 
 0.3 
 0.249178 
 -0.008138 
 
 
 0.45 
 nan 
 nan 
 
 
 0.6 
 0.256674 
 -0.015257 
 
 
 0.2 
 0.3 
 0.256517 
 -0.014007 
 
 
 0.45 
 0.256752 
 -0.015882 
 
 
 0.6 
 0.256674 
 -0.015257 
 
 
 16 
 0.05 
 0.3 
 0.248717 
 -0.023835 
 
 
 0.45 
 0.262161 
 -0.015790 
 
 
 0.6 
 0.262150 
 -0.015753 
 
 
 0.1 
 0.3 
 0.253869 
 -0.016510 
 
 
 0.45 
 0.262161 
 -0.015790 
 
 
 0.6 
 0.262150 
 -0.015753 
 
 
 0.2 
 0.3 
 0.253869 
 -0.016510 
 
 
 0.45 
 0.262161 
 -0.015790 
 
 
 0.6 
 0.262161 
 -0.015790 
 
 
 25 
 0.05 
 0.3 
 0.262488 
 -0.011821 
 
 
 0.45 
 0.262488 
 -0.011821 
 
 
 0.6 
 0.262488 
 -0.011821 
 
 
 0.1 
 0.3 
 0.262488 
 -0.011821 
 
 
 0.45 
 0.262488 
 -0.011821 
 
 
 0.6 
 0.262488 
 -0.011821 
 
 
 0.2 
 0.3 
 0.262488 
 -0.011821 
 
 
 0.45 
 0.262488 
 -0.011821 
 
 
 0.6 
 0.262488 
 -0.011821 
 
 EIT Dual return_pu_risk_sample return_pu_risk_oot k pho nuh 
 
 12 
 0.05 
 0.3 
 0.226302 
 0.021254 
 
 
 0.45 
 0.234465 
 0.002583 
 
 
 0.6 
 0.239274 
 0.014183 
 
 
 0.1 
 0.3 
 0.220133 
 0.005450 
 
 
 0.45 
 0.235800 
 0.013134 
 
 
 0.6 
 0.235272 
 0.007848 
 
 
 0.2 
 0.3 
 0.232128 
 0.020081 
 
 
 0.45 
 0.237084 
 0.013650 
 
 
 0.6 
 0.230653 
 0.018711 
 
 
 16 
 0.05 
 0.3 
 0.293255 
 -0.007802 
 
 
 0.45 
 0.320574 
 -0.000169 
 
 
 0.6 
 0.320697 
 0.004846 
 
 
 0.1 
 0.3 
 0.290247 
 -0.007121 
 
 
 0.45 
 0.318697 
 -0.000035 
 
 
 0.6 
 0.321263 
 0.003490 
 
 
 0.2 
 0.3 
 0.277200 
 -0.009775 
 
 
 0.45 
 0.322793 
 -0.000624 
 
 
 0.6 
 0.320697 
 0.004846 
 
 
 25 
 0.05 
 0.3 
 0.443169 
 -0.004166 
 
 
 0.45 
 0.472376 
 0.006710 
 
 
 0.6 
 0.471041 
 0.006855 
 
 
 0.1 
 0.3 
 0.427428 
 -0.010742 
 
 
 0.45 
 0.472364 
 0.006713 
 
 
 0.6 
 0.472412 
 0.006701 
 
 
 0.2 
 0.3 
 0.448598 
 -0.002375 
 
 
 0.45 
 0.472388 
 0.006707 
 
 
 0.6 
 0.472400 
 0.006704

In [27]:
combined_k_pho_nuh=basic_k_pho_nuh.merge(dual_k_pho_nuh,on=["k","pho","nuh"],suffixes=("_basic","_dual"))

In [35]:
((combined_k_pho_nuh["return_pu_risk_sample_dual"]>combined_k_pho_nuh["return_pu_risk_sample_basic"])).mean()

0.6666666666666666

In [31]:
combined_k_pho_nuh.columns

Index(['return_pu_risk_sample_basic', 'return_pu_risk_oot_basic',
       'return_pu_risk_sample_dual', 'return_pu_risk_oot_dual'],
      dtype='object')

In [37]:
((combined_k_pho_nuh["return_pu_risk_oot_dual"]>combined_k_pho_nuh["return_pu_risk_oot_basic"])).mean()

0.9259259259259259

In [33]:
combined_k_pho_nuh

return_pu_risk_sample_basic  return_pu_risk_oot_basic  \
k  pho  nuh                                                           
12 0.05 0.30                     0.260423                 -0.002815   
        0.45                          NaN                       NaN   
        0.60                     0.256674                 -0.015257   
   0.10 0.30                     0.249178                 -0.008138   
        0.45                          NaN                       NaN   
        0.60                     0.256674                 -0.015257   
   0.20 0.30                     0.256517                 -0.014007   
        0.45                     0.256752                 -0.015882   
        0.60                     0.256674                 -0.015257   
16 0.05 0.30                     0.248717                 -0.023835   
        0.45                     0.262161                 -0.015790   
        0.60                     0.262150                 -0.015753   
   0.10 0.30                     0.253869                 -0.016510   
        0.45                     0.262161                 -0.015790   
        0.60                     0.262150                 -0.015753   
   0.20 0.30                     0.253869                 -0.016510   
        0.45                     0.262161                 -0.015790   
        0.60                     0.262161                 -0.015790   
25 0.05 0.30                     0.262488                 -0.011821   
        0.45                     0.262488                 -0.011821   
        0.60                     0.262488                 -0.011821   
   0.10 0.30                     0.262488                 -0.011821   
        0.45                     0.262488                 -0.011821   
        0.60                     0.262488                 -0.011821   
   0.20 0.30                     0.262488                 -0.011821   
        0.45                     0.262488                 -0.011821   
        0.60                     0.262488                 -0.011821   

              return_pu_risk_sample_dual  return_pu_risk_oot_dual  
k  pho  nuh                                                        
12 0.05 0.30                    0.226302                 0.021254  
        0.45                    0.234465                 0.002583  
        0.60                    0.239274                 0.014183  
   0.10 0.30                    0.220133                 0.005450  
        0.45                    0.235800                 0.013134  
        0.60                    0.235272                 0.007848  
   0.20 0.30                    0.232128                 0.020081  
        0.45                    0.237084                 0.013650  
        0.60                    0.230653                 0.018711  
16 0.05 0.30                    0.293255                -0.007802  
        0.45                    0.320574                -0.000169  
        0.60                    0.320697                 0.004846  
   0.10 0.30                    0.290247                -0.007121  
        0.45                    0.318697                -0.000035  
        0.60                    0.321263                 0.003490  
   0.20 0.30                    0.277200                -0.009775  
        0.45                    0.322793                -0.000624  
        0.60                    0.320697                 0.004846  
25 0.05 0.30                    0.443169                -0.004166  
        0.45                    0.472376                 0.006710  
        0.60                    0.471041                 0.006855  
   0.10 0.30                    0.427428                -0.010742  
        0.45                    0.472364                 0.006713  
        0.60                    0.472412                 0.006701  
   0.20 0.30                    0.448598                -0.002375  
        0.45                    0.472388                 0.006707  
        0.60                    0.472400          